<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pulp as pp
import numpy as np
import pandas as pd

- https://www.ethanrosenthal.com/2016/07/20/lets-talk-or/
- https://www.ethanrosenthal.com/2016/08/30/towards-optimal-personalization/
- https://multithreaded.stitchfix.com/blog/2018/06/21/constrained-optimization/

In [2]:
# quantity that the collar requires
# particular quantity of production that will minimize the total annual
# cost of setting up and carrying inventory, if produced in one production run
names = ['leather', 'metal', 'canvas', 'cost', 'profit', 'min_run_size']
collars = pd.DataFrame([(0.50, 0.25, 0.30, 26.00, 10.50, 30),
                        (0.30, 0.70, 1.20, 29.00, 12.00, 40),
                        (0.90, 0.60, 0.57, 22.00, 09.00, 25),
                        (1.10, 0.45, 0.98, 26.50, 11.50, 60),
                        (0.75, 0.95, 0.55, 20.00, 08.50, 50)],
                      columns=names)
collars.index.name = 'Dog collar type'
collars

,leather,metal,canvas,cost,profit,min_run_size
Dog collar type,,,,,,
0,0.50,0.25,0.30,26.0,10.5,30
1,0.30,0.70,1.20,29.0,12.0,40
2,0.90,0.60,0.57,22.0,9.0,25
3,1.10,0.45,0.98,26.5,11.5,60
4,0.75,0.95,0.55,20.0,8.5,50


In [3]:
quants = pd.DataFrame([400, 250, 300],
                      index=['leather', 'metal', 'canvas'],
                      columns=['max_quantity'])
quants

,max_quantity
leather,400
metal,250
canvas,300


In [4]:
p = collars.profit
w = collars.cost
r = collars.min_run_size
m = quants.max_quantity
s = collars[['leather', 'metal', 'canvas']]
collar_index = range(collars.shape[0]) # j
material_index = range(s.shape[1]) # i
budget = 10000.0

In [5]:
prob = pp.LpProblem('Dog Collar Problem', pp.LpMaximize)

# Collar counts (the variable in our problem)
c = []
for j in collar_index:
    max_count = np.floor(budget / w[j])
    c.append(pp.LpVariable('c_{}'.format(j), lowBound=0, upBound=max_count, cat='Integer'))

# For pulp, add the objective function first
prob += pp.lpSum([i * j for i, j in zip(p, c)]), 'Total profit'
prob

Dog Collar Problem:
MAXIMIZE
10.5*c_0 + 12.0*c_1 + 9.0*c_2 + 11.5*c_3 + 8.5*c_4 + 0.0
VARIABLES
0 <= c_0 <= 384 Integer
0 <= c_1 <= 344 Integer
0 <= c_2 <= 454 Integer
0 <= c_3 <= 377 Integer
0 <= c_4 <= 500 Integer

In [6]:
# Budget constraint
prob += pp.lpSum([w[j] * c[j] for j in collar_index]) <= budget, 'Budget'

# Min run size constraint
for j in collar_index:
    prob += c[j] >= r[j], 'MinBatchSize_{}'.format(j)
    
# Max supplies quantity
for i in material_index:
    prob += pp.lpSum([s.iloc[j, i] * c[j] for j in collar_index]) <= m[i]

prob

Dog Collar Problem:
MAXIMIZE
10.5*c_0 + 12.0*c_1 + 9.0*c_2 + 11.5*c_3 + 8.5*c_4 + 0.0
SUBJECT TO
Budget: 26 c_0 + 29 c_1 + 22 c_2 + 26.5 c_3 + 20 c_4 <= 10000

MinBatchSize_0: c_0 >= 30

MinBatchSize_1: c_1 >= 40

MinBatchSize_2: c_2 >= 25

MinBatchSize_3: c_3 >= 60

MinBatchSize_4: c_4 >= 50

_C1: 0.5 c_0 + 0.3 c_1 + 0.9 c_2 + 1.1 c_3 + 0.75 c_4 <= 400

_C2: 0.25 c_0 + 0.7 c_1 + 0.6 c_2 + 0.45 c_3 + 0.95 c_4 <= 250

_C3: 0.3 c_0 + 1.2 c_1 + 0.57 c_2 + 0.98 c_3 + 0.55 c_4 <= 300

VARIABLES
0 <= c_0 <= 384 Integer
0 <= c_1 <= 344 Integer
0 <= c_2 <= 454 Integer
0 <= c_3 <= 377 Integer
0 <= c_4 <= 500 Integer

In [22]:
%timeit prob.solve(pp.COIN_CMD())

17.8 ms ± 410 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit prob.solve(pp.GLPK_CMD())

12.2 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
print("Status: {}".format(pp.LpStatus[prob.status]))

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print("{} = {} ".format(v.name, v.varValue))

# The optimised objective function value is printed to the screen
print('Total profit = ${:6.2f}'.format(pp.value(prob.objective)))
print('Total cost = ${:6.2f}'.format(np.sum(x * v.varValue for x, v in zip(w, prob.variables()))))

Status: Optimal
c_0 = 68.0 
c_1 = 40.0 
c_2 = 25.0 
c_3 = 151.0 
c_4 = 126.0 
Total profit = $4226.50
Total cost = $9999.50


In [9]:
# pip install pyomo
# brew install glpk
# https://projects.coin-or.org/Cbc

In [10]:
import numpy as np


stylist_workloads = {
    "Alex": 1, "Jennifer": 2, "Andrew": 2,
    "DeAnna": 2, "Jesse": 3
}

clients = [
    "Trista", "Meredith", "Aaron", "Bob", "Jillian",
    "Ali", "Ashley", "Emily", "Desiree", "Byron"
]

happiness_prob = {
    (stylist, client): np.random.rand()
    for stylist in stylist_workloads
    for client in clients
}
happiness_prob[('Alex', 'Trista')]

0.4749998630286989

In [11]:
from pyomo import environ as pe

model = pe.ConcreteModel()

In [12]:
model.stylists = pe.Set(initialize=stylist_workloads.keys())
model.clients = pe.Set(initialize=clients)

In [13]:
model.happiness_prob = pe.Param(
    # On pyomo Set objects, the '*' operator returns the cartesian product
    model.stylists * model.clients,
    # The dictionary mapping (stylist, client) pairs to chances of a happy outcome
    initialize=happiness_prob,
    # Happiness probabilities are real numbers between 0 and 1
    within=pe.UnitInterval)

model.stylist_workloads = pe.Param(
    model.stylists,
    initialize=stylist_workloads,
    within=pe.NonNegativeIntegers)

In [14]:
model.assignments = pe.Var(
    # Defined over the client-stylist matrix
    model.stylists * model.clients,
    # Possible values are 0 and 1
    domain=pe.Binary)

In [15]:
model.objective = pe.Objective(
    expr=pe.summation(model.happiness_prob, model.assignments),
    sense=pe.maximize)

In [16]:
def respect_workload(model, stylist):
    # Count up all the clients assigned to the stylist
    n_clients_assigned_to_stylist = sum(
        model.assignments[stylist, client]
        for client in model.clients)
    # What's the max number of clients this stylist can work with?
    max_clients = model.stylist_workloads[stylist]
    # Make sure that sum is no more than the stylist's workload
    return n_clients_assigned_to_stylist <= max_clients

model.respect_workload = pe.Constraint(
    # For each stylist in the set of all stylists...
    model.stylists,
    # Ensure that total assigned clients at most equal workload!
    rule=respect_workload)


def one_stylist_per_client(model, client):
    # Count up all the stylists assigned to the client
    n_stylists_assigned_to_client = sum(
        model.assignments[stylist, client]
        for stylist in model.stylists)
    # Make sure that sum is equal to one
    return n_stylists_assigned_to_client == 1

model.one_stylist_per_client = pe.Constraint(
    # For each client in the set of all clients...
    model.clients,
    # Ensure that exactly one stylist is assigned!
    rule=one_stylist_per_client)

In [17]:
solver = pe.SolverFactory("glpk")
# Add the keyword arg tee=True for a detailed trace of the solver's work.
solution = solver.solve(model)

In [18]:
assignments = model.assignments.get_values().items()
for (stylist, client), assigned in sorted(assignments):
    if assigned == 1:
        print("{} will be styled by {}".format(client.rjust(8), stylist))

   Byron will be styled by Alex
   Aaron will be styled by Andrew
 Desiree will be styled by Andrew
     Ali will be styled by DeAnna
  Trista will be styled by DeAnna
  Ashley will be styled by Jennifer
   Emily will be styled by Jennifer
     Bob will be styled by Jesse
 Jillian will be styled by Jesse
Meredith will be styled by Jesse


In [19]:
import numpy as np


stylist_workloads = {
    "Alex": 1, "Jennifer": 2, "Andrew": 2,
    "DeAnna": 2, "Jesse": 3
}

clients = [
    "Trista", "Meredith", "Aaron", "Bob", "Jillian",
    "Ali", "Ashley", "Emily", "Desiree", "Byron"
]

happiness_prob = {
    (stylist, client): np.random.rand()
    for stylist in stylist_workloads
    for client in clients
}
happiness_prob[('Alex', 'Trista')]

0.031613585132854505

In [20]:
# pip install pyomocontrib_simplemodel
from pyomo.contrib.simplemodel import SimpleModel, Binary, value

v = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
limit = 14
items = list(sorted(v.keys()))

# Create model
m = SimpleModel(maximize=True)

# Variables
x = m.var('m', items, within=Binary)

# Objective
m += sum(v[i]*x[i] for i in items)

# Constraint
m += sum(w[i]*x[i] for i in items) <= limit


# Optimize
status = m.solve('glpk')

# Print the status of the solved LP
print("Status = %s" % status.solver.termination_condition)

# Print the value of the variables at the optimum
for i in items:
    print("%s = %f" % (x[i], value(x[i])))

# Print the value of the objective
print("Objective = %f" % value(m.objective()))

Status = optimal
m[hammer] = 1.000000
m[screwdriver] = 1.000000
m[towel] = 1.000000
m[wrench] = 0.000000
Objective = 25.000000
